In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

import os

import glob
import sqlalchemy as sa
from sqlalchemy.orm import Mapped, relationship, DeclarativeBase, mapped_column, Session

from opengsync_db.models.Base import Base
from opengsync_db import categories, DBHandler, models
from opengsync_db import units

from flask_bcrypt import Bcrypt

from loguru import logger

from dotenv import load_dotenv
import os
load_dotenv()

bcrypt = Bcrypt()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
os.environ["POSTGRES_SERVER_IP"]

'lbi-dev.int.cemm.at'

In [10]:
db = DBHandler(expire_on_commit=False)
# db.connect(os.environ["POSTGRES_USER"], os.environ["POSTGRES_PASSWORD"], "localhost", port=5432)
db.connect(
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_SERVER_IP"],
    port=os.environ["POSTGRES_PORT"],
    db=os.environ["POSTGRES_DB"],
)

LOG: Connection closed.
LOG: Connected to DB 'postgresql+psycopg://lbi-dev.int.cemm.at:5432/opengsync_db'


In [11]:
label_search_columns: dict[str, list[str]] = {
    str(models.Project.__tablename__): ["identifier", "title"],
    str(models.SeqRequest.__tablename__): ["name"],
    str(models.Library.__tablename__): ["name"],
    str(models.Pool.__tablename__): ["name"],
    str(models.Experiment.__tablename__): ["name"],
    str(models.SeqRun.__tablename__): ["experiment_name"],
    str(models.Kit.__tablename__): ["name", "identifier"],
    str(models.User.__tablename__): ["email", "last_name", "first_name"],
    str(models.Group.__tablename__): ["name"],
    str(models.Feature.__tablename__): ["name", "target_name", "target_id"],
    str(models.SeqRun.__tablename__): ["experiment_name"],
    str(models.Plate.__tablename__): ["name"],
    str(models.Barcode.__tablename__): ["sequence", "name"],
}

In [12]:
q = """
SELECT * FROM pg_catalog.pg_tables;
"""
df = pd.read_sql(q, db._engine)

os.makedirs("init", exist_ok=True)

with open(os.path.join("init", "tables.txt"), "w") as f:
    for table in Base.metadata.tables.items():
        table_name = table[0]
        f.write(f"{table_name}\n")
        if table_name not in df["tablename"].values:
            raise Exception(f"Table {table[0]} is missing from the DB.")

        print(table_name)
        for column in table[1].columns:
            column_name = column.name
            f.write(f" - {column_name}\n")
            print(f" - {column_name}")
            q = f"SELECT column_name FROM information_schema.columns WHERE table_name='{table_name}' and column_name='{column_name}';"
            if len(pd.read_sql(q, db._engine)) == 0:
                raise Exception(f"Column {column_name} is missing from the table {table_name}.")

project
 - id
 - identifier
 - title
 - description
 - timestamp_created_utc
 - status_id
 - num_samples
 - owner_id
 - group_id
 - software
sample
 - id
 - name
 - status_id
 - num_libraries
 - qubit_concentration
 - avg_fragment_size
 - timestamp_stored_utc
 - project_id
 - ba_report_id
 - owner_id
 - attributes
user_affiliation
 - user_id
 - group_id
 - affiliation_type_id
sample_plate_link
 - plate_id
 - well_idx
 - sample_id
 - library_id
sample_library_link
 - mux
 - sample_id
 - library_id
lane_pool_link
 - lane_id
 - pool_id
 - experiment_id
 - dilution_id
 - num_m_reads
 - lane_num
library_feature_link
 - library_id
 - feature_id
seq_request_delivery_email_link
 - seq_request_id
 - email
 - status_id
experiment
 - id
 - name
 - timestamp_created_utc
 - timestamp_finished_utc
 - r1_cycles
 - r2_cycles
 - i1_cycles
 - i2_cycles
 - workflow_id
 - status_id
 - operator_id
 - sequencer_id
pool
 - id
 - name
 - status_id
 - type_id
 - timestamp_stored_utc
 - clone_number
 - original

In [13]:
# Extensions
q = """
SELECT * FROM pg_extension WHERE extname = 'pg_trgm';
"""

print("Checking if pg_trgm extension is installed.")
extensions = pd.read_sql(q, db._engine)
if len(extensions) == 0:
    print("Installing pg_trgm extension.")

    with db._engine.connect() as conn:
        conn.execute(sa.text('CREATE EXTENSION pg_trgm;COMMIT;'))
else:
    print("pg_trgm extension is installed.")

pd.read_sql(q, db._engine)

Checking if pg_trgm extension is installed.
pg_trgm extension is installed.


,oid,extname,extowner,extnamespace,extrelocatable,extversion,extconfig,extcondition
0,16981,pg_trgm,10,2200,True,1.6,None,None


In [14]:
# Indexes

# Postgres full text search
for table, columns in label_search_columns.items():
    for column in columns:
        with db._engine.connect() as conn:
            conn.execute(sa.text(f"""
                CREATE INDEX IF NOT EXISTS
                    trgm_{table}_{column}_idx
                ON
                    "{table}"
                USING
                    gin (lower({column}) gin_trgm_ops);COMMIT;
            """))

with db._engine.connect() as conn:
    conn.execute(sa.text("""
        CREATE INDEX IF NOT EXISTS
            trgm_user_full_name_idx
        ON
            "lims_user"
        USING
            gin ((first_name || ' ' || last_name) gin_trgm_ops);COMMIT;
    """))

    conn.execute(sa.text("""
        CREATE INDEX IF NOT EXISTS
            trgm_kit_identifier_name_idx
        ON
            "kit"
        USING
            gin ((identifier || ' ' || name) gin_trgm_ops);COMMIT;
    """))
